## About the data set
We have customer infromation which we got it from bank, and we have to predict base on customer credit score, salary etc whether customer will leave the bank in the comming years or will the cutomer stay and utilize the sevice of the bank.

Download the dataset from her https://www.kaggle.com/mathchi/churn-for-bank-customers

# Load Library

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

import matplotlib.pyplot as plt

from tensorflow import keras

# Load Dataset

In [ ]:
cust = pd.read_csv('../input/churn-for-bank-customers/churn.csv')

In [ ]:
# print the first five rows of the data set

cust.head()

In [ ]:
# print the shape of the dataset

cust.shape

In [ ]:
# few detail about data

cust.info()

In [ ]:
# statistical detail about data

cust.describe()

In [ ]:
# check for missing values

cust.isna().sum()

# Split the data

In [ ]:
# we wont take the first two columns (row id and customer id)

x = cust.iloc[:, 3:13]

# target variable
y = cust.iloc[:, -1]

# Convert categorical column to numerical

In [ ]:
x.Gender = pd.factorize(x.Gender)[0]

x.Geography = pd.factorize(x.Geography)[0]

In [ ]:
x.head()

# Train test split

In [ ]:
# Splitting the dataset into the Training set and Test set

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
display(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# Scaling the dataset

In [ ]:
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

# Atrifical Neural Network (ANN)

# Import Deep Learning Libraries

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, PReLU, ELU, Dropout

In [ ]:
# Initialising the ANN

classify = Sequential()

In [ ]:
# Adding the input layer and the first hidden layer

classify.add(Dense(units = 6, kernel_initializer = 'he_uniform', activation ='relu', input_dim =10))

# Adding the second hidden layer

classify.add(Dense(units = 6, kernel_initializer ='he_uniform' ,activation ='relu'))

# Adding the output layer

classify.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

In [ ]:
classify.summary()

In [ ]:
# Compiling the ANN

classify.compile(optimizer = 'adam', loss='binary_crossentropy', metrics='accuracy')

In [ ]:
# Fitting the ANN to the Training set

model = classify.fit(x_train, y_train, validation_split=0.20, batch_size=10, epochs=100)

In [ ]:
# list all data in histroy

print(model.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(model.history['accuracy'])
plt.plot(model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(model.history['loss'])
plt.plot(model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()


# Making predictions and evaluating model

In [ ]:
y_predict = classify.predict(x_test)
y_predict = (y_predict > 0.5)


print(confusion_matrix(y_test, y_predict))

print('\nAccuracy: ', accuracy_score(y_predict,y_test))

# Model using Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
lg = LogisticRegressionCV(cv=3)

lg.fit(x_train, y_train)

p_predict_train = lg.predict(x_train)

p_predict_test = lg.predict(x_test)

print(accuracy_score(p_predict_train, y_train))
print(accuracy_score(p_predict_test, y_test))

# Hyperparameter tuning

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import LeakyReLU, Activation, Dropout, Dense, Embedding, Flatten, BatchNormalization
from keras.activations import relu, sigmoid

In [ ]:
def create_model(layers, activation):
    model = Sequential()
    
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=x_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
layers = [(20,), (30, 20), (45, 30, 15)]
activations = ['sigmoid', 'relu']

param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[100])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(x_train, y_train)

In [ ]:
print(grid_result.best_score_,grid_result.best_params_)